In [2]:
!pip install pyforest

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=984143933a3cf28f4a82302047eb84e0b49aa97eb4cddd0b88bef0dae926f6eb
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


In [3]:
# importing libraries
from pyforest import *
# seeing all the libraries that were imported
lazy_imports()
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('/content/drive/MyDrive/CRedit Card Lead Prediction/train.csv')
df.head()

<IPython.core.display.Javascript object>

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [5]:
# dropping ID column as it would be unique for each customer and it would give no further knowledge to ML
df.drop(['ID'], axis=1, inplace=True)
df.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [6]:
# seperating independent and dependent features
x = df.iloc[:, :-1]
y = df['Is_Lead']
x.shape, y.shape

((245725, 9), (245725,))

In [7]:
test = pd.read_csv('/content/drive/MyDrive/CRedit Card Lead Prediction/test.csv')
test.head()

<IPython.core.display.Javascript object>

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [8]:
# dropping ID column as it would be unique for each customer and it would give no further knowledge to ML
test.drop(['ID'], axis=1, inplace=True)
test.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,Male,29,RG254,Other,X1,25,Yes,742366,No
1,Male,43,RG268,Other,X2,49,NaN,925537,No
2,Male,31,RG270,Salaried,X1,14,No,215949,No
3,Male,29,RG272,Other,X1,33,No,868070,No
4,Female,29,RG270,Other,X1,19,No,657087,No


In [10]:
!pip install feature-engine

     |████████████████████████████████| 153kB 6.2MB/s 
     |████████████████████████████████| 9.5MB 7.7MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [11]:
from feature_engine.imputation import CategoricalImputer

# we call the imputer from featur- engine
# we don't need to specify anything 

imputer = CategoricalImputer()

imputer.fit(x)

CategoricalImputer(fill_value='Missing', imputation_method='missing',
                   return_object=False,
                   variables=['Gender', 'Region_Code', 'Occupation',
                              'Channel_Code', 'Credit_Product', 'Is_Active'])

In [12]:
x = imputer.transform(x)
test = imputer.transform(test)

In [13]:
# label encoding gender, credit_product and is_Active
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# le.fit(df['Gender'])
# le.fit(df['Credit_Product'])
# le.fit(df['Is_Active'])


x['Gender'] = le.fit_transform(x['Gender'])
x['Credit_Product'] = le.fit_transform(x['Credit_Product'])
x['Is_Active'] = le.fit_transform(x['Is_Active'])
x['Region_Code'] = le.fit_transform(x['Region_Code'])
x['Occupation'] = le.fit_transform(x['Occupation'])
x['Channel_Code'] = le.fit_transform(x['Channel_Code'])

test['Gender'] = le.fit_transform(test['Gender'])
test['Credit_Product'] = le.fit_transform(test['Credit_Product'])
test['Is_Active'] = le.fit_transform(test['Is_Active'])
test['Region_Code'] = le.fit_transform(test['Region_Code'])
test['Occupation'] = le.fit_transform(test['Occupation'])
test['Channel_Code'] = le.fit_transform(test['Channel_Code'])

x.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,0,73,18,1,2,43,1,1045696,0
1,0,30,27,2,0,32,1,581988,0
2,0,56,18,3,2,26,1,1484315,1
3,1,34,20,2,0,19,1,470454,0
4,0,30,32,2,0,33,1,886787,0


In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=4)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((164635, 9), (81090, 9), (164635,), (81090,))

In [16]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_scaled = pd.DataFrame(mms.fit_transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(mms.transform(x_test), columns=x_test.columns)
test_scaled = pd.DataFrame(mms.transform(test), columns=test.columns)
# we have now fit and transform the data into a scaler for accurate reading and results.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_balanced, y_balanced = oversample.fit_resample(x_scaled, y_train)
x_test_balanced, y_test_balanced = oversample.fit_resample(x_test_scaled, y_test)
# we have addressed the issue of oversampling here

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [19]:
classifiers = {
    "LogisticRegression" : LogisticRegression(),
    "KNeighbors" : KNeighborsClassifier(),
    "SVC" : SVC(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "XGBoost" : XGBClassifier()
}

In [20]:
train_scores = []
test_scores = []

for key, classifier in classifiers.items():
    classifier.fit(x_balanced, y_balanced)
    train_score = classifier.score(x_balanced, y_balanced)
    train_scores.append(train_score)
    test_score = classifier.score(x_test_balanced, y_test_balanced)
    test_scores.append(test_score)

print(train_scores)
print(test_scores)

[0.6720796019900498, 0.893273631840796, 0.7929432835820895, 0.9999960199004975, 0.999976119402985, 0.8314189054726369]
[0.6735908885006148, 0.7575794344140296, 0.7927910438102633, 0.823351452792338, 0.8510078949071378, 0.8351776354105999]


In [23]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

rf.fit(x_train, y_train)

rf_accuracies = cross_val_score(rf, X=x_test, y=y_test, cv=10)

print('Accuracies" \n', rf_accuracies)
print()
print('Mean Accuracy: ', rf_accuracies.mean())

Accuracies" 
 [0.85065976 0.85509927 0.85460599 0.85041312 0.86237514 0.86015538
 0.85472931 0.85065976 0.85657911 0.8521396 ]

Mean Accuracy:  0.8547416450857073
